In [1]:
import pandas as pd
import networkx as nx
import numpy as np 
import json
print(nx.__version__)

import plotly.plotly as py
from plotly.graph_objs import *

from IPython.display import display

2.1


In [2]:
def fragments_to_graph(fname, kmer=5):
    file = pd.read_csv(fname)
    display(file.head())
    molecules = file['Molecule'].as_matrix()
    starts = file['Start'].as_matrix()
    ends = file['End'].as_matrix()
    queries = file['Query'].as_matrix()
    
    kmer_dict = {}
    
    G = nx.DiGraph()
    
    for (i, molecule) in enumerate(molecules):
        frag = molecule[starts[i]:ends[i]]
        for j in range(starts[i],ends[i]-kmer):
            origin = '{0}_{1}'.format(j,molecule[j:j+kmer])
            dest = '{0}_{1}'.format(j+1,molecule[j+1:j+kmer+1])
            
            if (origin, dest) in G.edges():
                data = G.get_edge_data(origin, dest)
                G.add_edge(origin, dest, capacity=data['capacity']+1)
            else:
                G.add_edge(origin, dest, capacity=1)
            
            try: 
                kmer_dict[origin].append(dest)
            except:
                kmer_dict[origin] = [dest]
    print('Created graph out of {0} molecules'.format(len(molecules)))            

    return G, kmer_dict

def condense(oldG):
    
    import copy
    G = copy.deepcopy(oldG)
    condense = [(None,None,None)]
    init = True

    while len(condense)!=0:
        (origin, dest, capacity) = condense.pop()
        if origin != None:
            hybrid = origin+dest[-1]
            # All edges going in to origin now go to the hybrid edge        
            removals = []
            for node, _, capacity in G.in_edges(origin, data='capacity'):
                removals.append((node, capacity))
            for (node,capacity) in removals:
                G.remove_edge(node, origin)
                G.add_edge(node, hybrid, capacity=capacity)   

            # All edges going out of dest now come from the hybrid edge
            removals = []
            for _, node, capacity in G.out_edges(dest, data='capacity'):
                removals.append((node, capacity))
            for (node, capacity) in removals:
                G.remove_edge(dest, node)
                G.add_edge(hybrid, node, capacity=capacity)

            # Remove edge from origin to dest
            G.remove_edge(origin, dest)

            G.remove_node(origin)
            G.remove_node(dest)
        for node, out_degree in G.out_degree():
            if out_degree == 1:
                #Condense unambiguous paths
                dest = list(G[node])[0]
                condense.append((node,dest,G[node][dest]))
                break
    return G

def anchor_ends(oldG):
    import copy
    G = copy.deepcopy(oldG)
    edges = []
    for node, in_degree in G.in_degree():
        if in_degree == 0:
            edges.append(('SOURCE',node))
    for node, out_degree in G.out_degree():
        if out_degree == 0:
            edges.append((node,'SINK'))
    for edge in edges:
        G.add_edge(*edge)

    return G
    
def assembly_graph(fragments_fname, kmer=30):
    ''' Wrapper for creating an assembly graph from sequenced fragments'''
    
    G, kmer_dict = fragments_to_graph(fragments_fname, kmer=kmer)
    anchoredG = anchor_ends(G)
    nx.write_gexf(G, "basic_assembly_"+fragments_fname[:-4]+'.gexf')  
    conG = condense(G)
    nx.write_gexf(G, "condensed_assembly_"+fragments_fname[:-4]+'.gexf')
    
    print('Saved graphs to .gexf')
    return anchoredG, conG

In [5]:
anchoredG, conG = assembly_graph('LongTestFile_1_basevector_PCR_frag.csv',30)

,Unnamed: 0,Amplicon,Query,Molecule,Start,End,count
0,0,2,1,.................................................,261,860,1
1,1,8,6,0000000000000000000000000000000000000000000000...,0,578,1
2,2,9,7,0000000000000000000000000000000000000000000000...,0,509,1
3,3,11,9,0000000000000000000000000000000000000000000000...,0,534,1
4,4,15,11,.................................................,407,987,1


Created graph out of 552 molecules
Saved graphs to .gexf


In [ ]:
def plot_graph(G):
    plt.figure(figsize=(15,15))
    edge_labels=dict([((u,v,),d)
                     for u,v,d in G.edges(data='capacity')])
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_edge_labels(G,pos, edge_labels=edge_labels)
    nx.draw(G,pos,with_labels=True)
    plt.show()
    plt.close()
    
plot_graph(conG)

In [ ]:
plot_graph(G)

In [ ]:
flow, path = nx.algorithms.flow.maximum_flow(anchoredG, 'SOURCE','SINK')

In [ ]:
path

'''
How do we reconstruct the original reads from the capacities assigned to max flow graphs?

'''
def reconstruct_reads_from_path():
    for edge in path:
        pass
    

In [ ]:
for origin in path:
    weights = path[origin]
    for dest in weights:
        anchoredG[origin][dest]['weight']=weights[dest]

In [ ]:
plot_graph(anchoredG)

In [ ]:
path

In [ ]:
nx.write_gexf(anchoredG, "max_flow_assembly.gexf")

In [ ]:
anchoredG

In [ ]:
for edge in anchoredG.edges(data=True):
    print(edge)

In [ ]:
nx.write_gexf(anchoredG, "max_flow_assembly.gexf")

In [ ]:
path = nx.algorithms.flow.max_flow_min_cost(anchoredG,'SOURCE','SINK')

In [ ]:
resG = path

In [ ]:
for edge in resG.edges(data=True):
    print(edge)

In [ ]:
resG

In [ ]:
def reweight(G,path):
    G = copy.deepcopy(G)
    for origin in path:
        weights = path[origin]
        for dest in weights:
            G[origin][dest]['weight']=weights[dest]
    return G

In [ ]:
nx.write_gexf(anchoredG, "max_flow_assembly.gexf")

In [ ]:
conG = condense(anchoredG)

In [ ]:
describe(G)

In [ ]:
for edge in conG.edges(data=True):
    print(edge)